In [7]:
import pandas as pd

main_file_path = '../input/train.csv'
data = pd.read_csv(main_file_path)
print(data.describe())



In [11]:
iowa_price_data = data.SalePrice

print(iowa_price_data.describe())
print(iowa_price_data.head())

In [19]:
iowa_price_lot_columns = ['SalePrice','LotArea']
iowa_price_lot = data[iowa_price_lot_columns]
print(iowa_price_lot.describe())

In [22]:
y = iowa_price_data
#determining predictors
iowa_predictors = ['LotArea','YearBuilt','1stFlrSF','2ndFlrSF','FullBath','BedroomAbvGr','TotRmsAbvGrd']
X = data[iowa_predictors]
#importing decision tree
from sklearn.tree import DecisionTreeRegressor
iowa_model = DecisionTreeRegressor()
#determining fit for model
iowa_model.fit(X,y)

#printing var for the first 5 houses
print(X.head())
print('For these 5 houses, their predicted prices are..')
#printing predictions
print(iowa_model.predict(X.head()))

In [24]:
#finding MAE
iowa_predicted_prices = iowa_model.predict(X)
#import MAE
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y,iowa_predicted_prices)

#but this is wrong as we are validating the prediciton with data we used for prediction

In [25]:
#we now break up data to reserve one for validation
from sklearn.model_selection import train_test_split
#we split into 2 var, train and val. Could include more if needed
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)
iowa_model = DecisionTreeRegressor()
iowa_model.fit(train_X,train_y)
#based on iowa_model that was trained using training data, i predict using validation data
val_predictions = iowa_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

In [32]:
#now we compare number of leaves in a decision tree to determing the best possible number of leaves
#so here we def a function get_mae to make it easier
def get_mae(max_leaf_nodes, predictors_train, predictors_val, targ_train, targ_val):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(predictors_train, targ_train)
    preds_val = model.predict(predictors_val)
    mae = mean_absolute_error(targ_val, preds_val)
    return(mae)

#using a for function, we run get_mae multiple times to get values of MAE to compare
for i in [5,50,500,5000]:
    my_mae = get_mae(i,train_X,val_X,train_y,val_y) 
    #\t just represents indent in normal documents
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(i, my_mae))
    #from data it seems 50 nodes is the best

What we're doing next is to insteas utilise random forest to generate better predictions based on exisiting data we used in the first ML model

In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor()
forest_model.fit(train_X, train_y)
iowa_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, iowa_preds))

Now we test this model utilising the provided test data

In [40]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
# Read the test data
test = pd.read_csv('../input/test.csv')

test_X = test[iowa_predictors]
predicted_prices = iowa_model.predict(test_X)
print(predicted_prices)

In [42]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)